### Imports modules and libs

In [ ]:
import pandas as pd 
import numpy as np
from ConvertFile import Convert_File_to_pd
from DataAnalyze import DataAnalyze
from AnalyzeDuplicatedData import AnalyzeDuplicatedData
from DataTransform import DataTransform
import re 
from cmath import isnan
from ObsMatchFields import ObsMatchFields

### Transform CSVs to DFs

In [ ]:
Data_frame=Convert_File_to_pd(file_name='Match',name_table='Data',folder_name='ETL-Data-FIFA')

In [ ]:
Df=Data_frame.convert_to_dataframe

### Exploratory analysis to investigate the possibility of data cleaning

In [ ]:
#Imprimindo o DataFrame
Df

#### Premises

In [ ]:
#Como são 115 colunas, é necessário prioriza-las 

for col in Df.columns:
    print(col,end=' , ')

In [ ]:
#Para dar prioridade, os campos dos players, assim como as siglas contendo as 3 letras maiusculas serão removidos, pois tem campos acima que retraram 
#estatisticas de partidas, assim como os ids que irão servir para referenciar outras tabelas 

#Para o caso dos players, criou-se dois vetores que logicamente terão os mesmos tamanhos, já que o número dos jogadores de casa devem ser os mesmos dos visitantes
ls_col=Df.columns.tolist()
arra_home_p=list(Df.columns.str.startswith('home_player_'))
arra_away_p=list(Df.columns.str.startswith('away_player_'))

#Olhando para lista acima, percebe-se que o padrão é de 3 letras maisculas ou 4 com 365
#Seria possível de fazer uma reg expression para detectar, mas para ficar mais simples é só identificar o indice do B365H, que depois dele todos serão evitados
#Como existem Falses nas condições lógicas abaixo, esse vetor original considerado sera deslocado, logo subtraindo os Trues, teremos o vetor filtrado

ls_fin=[ls_col[a] for a in range(len(arra_home_p)) if arra_home_p[a]==False and arra_away_p[a]==False][:ls_col.index('B365H')-sum(arra_home_p+arra_away_p)]


In [ ]:
dici_updated={}

for s in ls_fin:
    v=Df[s]
    dici_updated[s]=v

Df=pd.DataFrame(dici_updated)


In [ ]:
#Classe responsável pelas análises exploratórias
Results_analysis=DataAnalyze(Df)

#### Detect NaNs

In [ ]:
#Avaliando a proporção de NaNs em todos os campos
Results_analysis.count_nans(name_columns=list(Df.columns))


##### Observations

In [ ]:
#Percebe-se que as ultimas colunas referentes aos acontecimentos da partida tem o mesmo valor da proporção de NaNs


In [ ]:
##visando investigar se os dados em não NaN são os mesmos
#Como a coluna goal já existe como info em outros campos anteriores, será avaliado para os demais campos
#Fora isso, pegaremos somente as 4 últimas colunas para não gerar ainda mais complexidade nas futuras inferências
Df.drop(['goal','shoton','shotoff'],axis=1,inplace=True)



dici_not_nans={}
r=list(Df.columns)[11:]
for t in r:
    temp=Df[t].notna()
    q=[Df[t][x] for x in range(temp.shape[0]) if temp[x]]
    dici_not_nans[t]=q


Df_temp=pd.DataFrame(dici_not_nans)



In [ ]:
Df_temp

##### criação de novas colunas com base nas antigas

###### COLUNA FOUL

In [ ]:
#Função para detectar a quantidade de ocorrencias de um time de casa ou time de fora

def cont_by_teams(Df,name_of_column,name_key_home_team,name_key_away_team):
    ds={}
    for z in range((Df.shape[0])):
        if isinstance(Df[name_of_column][z],str):
            home_team=Df['home_team_api_id'][z]
            away_team=Df['away_team_api_id'][z]
            
      

            rgxx_team="((<team>)[0-9]+(</team>))"

            string_aval=re.findall(rgxx_team,Df[name_of_column][z])
            
            
            cont_home,cont_away=0,0
            for s in string_aval:

                if s != None:
                    comp1=len(s[0])
                    value=s[0][len("<team>"):comp1-len("</team>")]
                   
                
                    if value == str(home_team): cont_home+=1
                    else: cont_away+=1
                
            ds[z]={name_key_home_team:cont_home,name_key_away_team:cont_away}
        
        else:
            ds[z]={name_key_home_team:0,name_key_away_team:0}


    return ds 

In [ ]:
#percebe-se abaixo que é possível extrair informações posteriores a respeito de tempo, o time em que executou algo e os jogadores
#Todavia, para não estabelecer mais complexidade à tabela, os dados serão ramificados em outra(s) colunas, seguindo as estratégias:


#Para coluna foulcommit, ramificação em duas colunas da quantidade de faltas cometidas pelo time da casa e pelo time visitante

In [ ]:
dict_foul=cont_by_teams(Df=Df,name_of_column='foulcommit',name_key_home_team="quant_home_team_foul_commit",name_key_away_team="quant_away_team_foul_commit")

In [ ]:
lis_foul=list(dict_foul.values())
home_list,away_list=[],[]

for i in lis_foul:
    home_list.append(i['quant_home_team_foul_commit'])
    away_list.append(i['quant_away_team_foul_commit'])

#gerar a lista contendo os valores dos times 
Df['quant_home_team_foul_commit']=home_list
Df['quant_away_team_foul_commit']=away_list


In [ ]:
#Dropar a coluna antiga
Df.drop(['foulcommit'],axis=1,inplace=True)

###### COLUNA CARD

In [ ]:
#Para coluna card, as colunas serão convertidas em contagem de cartões amarelos e vermelhos para os times da casa e de fora

ds_card={}

for z in range((Df.shape[0])):
    if isinstance(Df['card'][z],str):
        home_team=Df['home_team_api_id'][z]
        away_team=Df['away_team_api_id'][z]
        
        string_aval=Df['card'][z]

        rgxx="((<card_type>)[a-z-A-Z]+(<\/card_type>))"
        rgxx_team="((<team>)[0-9]+(<\/team>))"

        
        pat1=re.findall(rgxx_team,string_aval)
        pat2=re.findall(rgxx,string_aval)

        yhteam,yateam=0,0
        rhteam,rateam=0,0
        for patt1,patt2 in zip(pat1,pat2):
            if patt1!=None:

                #Para retirar o id do time no qual realizou a falta para cometer o cartão, o slice é feito a partir do comprimento da primeira tag (<team>)
                #na qual contem 6 caracteres, e a partir dele pegar o número(ID do time) até o comprimento - 7(valor de comprimento da tag </team>)
                #já que len(<tag>)+len(id_num)+len(</tag>)=len(tot)
                comp1=len(patt1[0])
                slice_pat1=patt1[0][len("<team>"):comp1-len("</team>")]

                #mesma logica para a tag card_type
                comp2=len(patt2[0])
                slice_pat2=patt2[0][len("<card_type>"):comp2-len("</card_type>")]

                if slice_pat2.lower() == 'y':
                    if slice_pat1==str(home_team):
                        yhteam+=1
                    else:yateam+=1

                else:
                    if slice_pat1==str(home_team):
                        rhteam+=1
                    else:rateam+=1
        
        
        ds_card[z]={"number_yellow_cards_home_team":yhteam,"number_yellow_cards_away_team":yateam,"number_red_cards_home_team":rhteam,"number_red_cards_away_team":rateam}

    else:
        ds_card[z]={"number_yellow_cards_home_team":0,"number_yellow_cards_away_team":0,"number_red_cards_home_team":0,"number_red_cards_away_team":0}




    

In [ ]:
lis_card=list(ds_card.values())
y_home_list,y_away_list,r_home_list,r_away_list=[],[],[],[]

for i in lis_card:
    y_home_list.append(i["number_yellow_cards_home_team"])
    y_away_list.append(i["number_yellow_cards_away_team"])
    r_home_list.append(i["number_red_cards_home_team"])
    r_away_list.append(i["number_red_cards_away_team"])

#gerar a lista contendo os valores dos times 
Df["number_yellow_cards_home_team"]=y_home_list
Df["number_yellow_cards_away_team"]=y_away_list
Df["number_red_cards_home_team"]=r_home_list
Df["number_red_cards_away_team"]=r_away_list


In [ ]:
#Dropar coluna antiga
Df.drop(['card'],axis=1,inplace=True)

###### COLUNA CROSS

In [ ]:
#Para a coluna cross, quantidade de cruzamentos do time da casa e de fora serão convertidos em colunas 

ds_cross={}


for z in range((Df.shape[0])):
    if isinstance(Df['cross'][z],str):
        home_team=Df['home_team_api_id'][z]
        away_team=Df['away_team_api_id'][z]
        

        rgxx_team="((<team>)[0-9]+(</team>))"

        var_dec=(Df['cross'][z].split("</value>"))
        
        cont_home_cross,cont_away_cross=0,0
        for s in var_dec:
            q=re.search(rgx,s)
            if q != None:
                
                q=q.span()
                u=(q[0]+len(("<team>")),q[1]-len(("</team>")))
                value=s[u[0]:u[1]]
            
                if value == str(home_team): cont_home_cross+=1
                else: cont_away_cross+=1
              
        ds_cross[z]={"quant_home_team_cross":cont_home_cross,"quant_away_team_cross":cont_away_cross}
       
    else:
        ds_cross[z]={"quant_home_team_cross":0,"quant_away_team_cross":0}

In [ ]:
lis_cross=list(ds_cross.values())
cross_home,cross_away=[],[]

for i in lis_cross:
   cross_home.append(i["quant_home_team_cross"])
   cross_away.append(i["quant_away_team_cross"])

#gerar a lista contendo os valores dos times 

Df["quant_home_team_cross"]=cross_home
Df["quant_away_team_cross"]=cross_away


In [ ]:
#Dropar coluna antiga
Df.drop(['cross'],axis=1,inplace=True)

###### COLUNA CORNER 

In [ ]:
ds_corner={}


for z in range((Df.shape[0])):
    if isinstance(Df['corner'][z],str):
        home_team=Df['home_team_api_id'][z]
        away_team=Df['away_team_api_id'][z]
        

        rgxx_team="((<team>)[0-9]+(</team>))"

        var_dec=(Df['corner'][z].split("</value>"))
        print(var_dec)
        break

#### Detect Data Duplicated

In [ ]:
#além disso, é necessário avaliar os registros duplicados quando a coluna não pode contê-los, como é o caso do ID também
Result_dup_id=AnalyzeDuplicatedData(dataframe=Df,name_col_data='id')


In [ ]:
#Avaliando a quantidade de ids duplicados
Result_dup_id.print_duplicated_data(table_name='Country')

In [ ]:
#Fazendo isso para coluna country_id e name
Result_dup_country_id=AnalyzeDuplicatedData(dataframe=Df,name_col_data='country_id')
Result_dup_name=AnalyzeDuplicatedData(dataframe=Df,name_col_data='name')


In [ ]:
Result_dup_country_id.print_duplicated_data(table_name='Country')
Result_dup_name.print_duplicated_data(table_name='Country')

#### Identify Outliers

In [ ]:
#Como a coluna ID e country_id são de identificação, não há valores que possam determinar um outlier, a não ser que eles fossem autoincrementais
#Porém, não é o caso, conforme pode ser visto acima no Df 



### Analyze  typing

In [ ]:
#analisando a tipagem esperada de forma geral
Df.info()


In [ ]:
#Como no método info a coluna name é object e object pode conter qualquer tipagem, então é necessário investigar se todas são Strings para serem Name
Results_analysis.confirm_typing(name_column='name',type_expected='str')

#Já os demais devem ser mantidos

### Data transform 

In [ ]:
#Como não houve acontecimento de NaNs, dados duplicados, outliers e tipagens fora do escopo, o Df já está devidamente tratado
Data_Transf=Df.copy()


In [ ]:
Data_Transf